In [1]:
import kotlin.io.path.Path
import kotlin.io.path.readLines



In [28]:
val inputStr = readInput("Day16_test")
inputStr.forEach { it.println() }


.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....


In [4]:
enum class Heading { UP, DOWN, LEFT, RIGHT }
data class Position(
    val x: Int,
    val y: Int,
)

fun Position.up() = this.copy(y = y - 1)
fun Position.down() = this.copy(y = y + 1)
fun Position.left() = this.copy(x = x - 1)
fun Position.right() = this.copy(x = x + 1)


data class Beam(
    var currentPosition: Position,
    var currentHeading: Heading,
    var isFinished: Boolean = false,
    val energizedTiles: MutableSet<Position> = mutableSetOf(),
)

In [5]:
val visited = mutableSetOf<Pair<Position, Heading>>()

fun Beam.move(input: List<String>): Beam? {
    if (isFinished) return null
    if (currentPosition.x >= input[0].length ||
        currentPosition.y >= input.size ||
        currentPosition.x < 0 || currentPosition.y < 0 ||
        (currentPosition to currentHeading) in visited
    ) {
        isFinished = true
        return null
    }
    visited.add(currentPosition to currentHeading)

    currentPosition = when (currentHeading) {
        Heading.UP -> currentPosition.up()
        Heading.DOWN -> currentPosition.down()
        Heading.RIGHT -> currentPosition.right()
        Heading.LEFT -> currentPosition.left()
    }

    energizedTiles.add(currentPosition)

    val currentInstruction = input[currentPosition.y][currentPosition.x]
    if (currentInstruction == '\\') {
        currentHeading = when (currentHeading) {
            Heading.UP -> Heading.LEFT
            Heading.LEFT -> Heading.UP
            Heading.DOWN -> Heading.RIGHT
            Heading.RIGHT -> Heading.DOWN
        }
    } else if (currentInstruction == '/') {
        currentHeading = when (currentHeading) {
            Heading.UP -> Heading.RIGHT
            Heading.LEFT -> Heading.DOWN
            Heading.DOWN -> Heading.LEFT
            Heading.RIGHT -> Heading.UP
        }
    } else if (currentInstruction == '|' && currentHeading in setOf(Heading.LEFT, Heading.RIGHT)) {
        currentHeading = Heading.UP
        return this.copy(energizedTiles = mutableSetOf(), currentHeading = Heading.DOWN)
    } else if (currentInstruction == '-' && currentHeading in setOf(Heading.UP, Heading.DOWN)) {
        currentHeading = Heading.RIGHT
        return this.copy(energizedTiles = mutableSetOf(), currentHeading = Heading.LEFT)
    }
    return null
}

//val beam = Beam(currentPosition = Position(0, 0), Heading.RIGHT).also { it.println() }
//val moved = beam.move(inputStr)!!.also { it.println() }
//moved.move(inputStr).println()
//moved.println()


In [29]:
data class PositionBeam(
    val position: Position,
    val heading: Heading
)

fun Position.isValid() = x < inputStr[0].length && x >= 0 && y < inputStr.size && y >= 0

val currentVisited = mutableSetOf<PositionBeam>()

fun search(currentBeam: PositionBeam): Set<PositionBeam> {

    val newPosition = when (currentBeam.heading) {
        Heading.UP -> currentBeam.position.up()
        Heading.DOWN -> currentBeam.position.down()
        Heading.RIGHT -> currentBeam.position.right()
        Heading.LEFT -> currentBeam.position.left()
    }
    if (!newPosition.isValid() || currentBeam in currentVisited) {
        return if (currentBeam.position.isValid()) setOf(currentBeam) else emptySet()
    }
    if (inputStr[newPosition.y].length <= newPosition.x) {
        println("${newPosition.y} seems invalid for position $newPosition: ${inputStr[newPosition.y]}")
    }
    val currentInstruction = inputStr[newPosition.y][newPosition.x]
    var newHeading = currentBeam.heading
    currentVisited.add(currentBeam)

    if (currentInstruction == '\\') {
        newHeading = when (newHeading) {
            Heading.UP -> Heading.LEFT
            Heading.LEFT -> Heading.UP
            Heading.DOWN -> Heading.RIGHT
            Heading.RIGHT -> Heading.DOWN
        }
    } else if (currentInstruction == '/') {
        newHeading = when (newHeading) {
            Heading.UP -> Heading.RIGHT
            Heading.LEFT -> Heading.DOWN
            Heading.DOWN -> Heading.LEFT
            Heading.RIGHT -> Heading.UP
        }
    } else if (currentInstruction == '|' && newHeading in setOf(Heading.LEFT, Heading.RIGHT)) {
        return search(PositionBeam(newPosition, Heading.UP)) +
                search(PositionBeam(newPosition, Heading.DOWN)) + currentBeam
    } else if (currentInstruction == '-' && newHeading in setOf(Heading.UP, Heading.DOWN)) {
        return search(PositionBeam(newPosition, Heading.LEFT)) +
                search(PositionBeam(newPosition, Heading.RIGHT)) + currentBeam
    }
    return search(PositionBeam(newPosition, newHeading)) + currentBeam
}

fun printPositions(positions: Set<Position>) {
    println()
    (0 until inputStr.size).forEach { y ->
        (0 until inputStr[0].length).forEach { x ->
            if (Position(x, y) in positions)
                print("#")
            else {
                print(".")
            }
        }
        println()
    }
}

val energizedPositions = search(PositionBeam(Position(3, -1), Heading.DOWN)).map { it.position }.toSet()
energizedPositions.forEach { it.println() }
//printPositions(energizedPositions)
energizedPositions.count() - 1


Position(x=0, y=7)
Position(x=1, y=7)
Position(x=2, y=7)
Position(x=3, y=7)
Position(x=1, y=8)
Position(x=1, y=9)
Position(x=2, y=8)
Position(x=3, y=8)
Position(x=4, y=8)
Position(x=5, y=8)
Position(x=6, y=8)
Position(x=1, y=0)
Position(x=1, y=6)
Position(x=1, y=5)
Position(x=1, y=4)
Position(x=1, y=3)
Position(x=1, y=2)
Position(x=1, y=1)
Position(x=2, y=0)
Position(x=3, y=0)
Position(x=4, y=0)
Position(x=5, y=0)
Position(x=5, y=1)
Position(x=5, y=2)
Position(x=5, y=9)
Position(x=5, y=7)
Position(x=5, y=6)
Position(x=5, y=5)
Position(x=5, y=4)
Position(x=5, y=3)
Position(x=6, y=2)
Position(x=9, y=2)
Position(x=8, y=2)
Position(x=7, y=2)
Position(x=6, y=3)
Position(x=6, y=4)
Position(x=6, y=5)
Position(x=6, y=6)
Position(x=7, y=6)
Position(x=7, y=7)
Position(x=7, y=8)
Position(x=7, y=9)
Position(x=6, y=7)
Position(x=4, y=6)
Position(x=4, y=7)
Position(x=3, y=6)
Position(x=3, y=5)
Position(x=3, y=4)
Position(x=3, y=3)
Position(x=3, y=2)
Position(x=3, y=1)
Position(x=3, y=-1)


51

In [25]:
fun initalBeams() =
    (0..inputStr.size).flatMap { y ->
        setOf(
            PositionBeam(Position(-1, y), Heading.RIGHT),
            PositionBeam(Position(inputStr.size, y), Heading.LEFT)
        )
    }.toSet() + (0..inputStr[0].length).flatMap { x ->
        setOf(
            PositionBeam(Position(x, -1), Heading.DOWN),
            PositionBeam(Position(x, inputStr[0].length), Heading.UP)
        )
    }.toSet()

printPositions(initalBeams().map { it.position }.toSet())
initalBeams().forEach { it.println() }
initalBeams().count()


Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................................................................................................Line_21_jupyter@1f3ec9f9
..............................

444

In [26]:
var iter = 0 
initalBeams()
    .asSequence()
    .map { currentVisited.clear(); search(it) }
    .map { it.map { it.position }.toSet().count() - 1 }
    .map { iter++;  println("Iteration $iter"); it }
    .max()

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration 77
Iteratio

7943